# Climate Change Impact on Agriculture — CRISP‑DM (Professional Colab)
**Scope:** End‑to‑end, compute‑aware CRISP‑DM walkthrough: EDA → Cleaning/Preprocessing → Feature Engineering (outlier flags) → Baselines & Models → Evaluation & Interpretability → Final Recommendation.  
**Packages:** `pandas`, `numpy`, `scikit‑learn`, `matplotlib`, `seaborn` (optional `shap`).

**Author’s note:** This notebook is designed for **Colab**. It includes upload/Drive/Kaggle options for data access and compute‑friendly toggles.

## Table of Contents
1. [Environment & Config](#env)  
2. [Data Access](#data)  
3. [CRISP‑DM Step 1 — Business Understanding](#bu)  
4. [CRISP‑DM Step 2 — Data Understanding (EDA)](#eda)  
5. [CRISP‑DM Step 3 — Data Preparation](#prep)  
6. [CRISP‑DM Step 4 — Baselines & Candidate Models](#models)  
7. [CRISP‑DM Step 5 — Evaluation & Interpretability](#eval)  
8. [CRISP‑DM Step 6 — Final Recommendation & Reporting](#final)  
9. [Artifacts & Download](#artifacts)

## 1) Environment & Config <a id='env'></a>

In [ ]:
#@title Setup & compute-aware config
import sys, os, math, warnings
import numpy as np
import pandas as pd

# Optional visualization
import matplotlib.pyplot as plt
import seaborn as sns

from typing import List, Tuple, Dict

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 160)
pd.options.display.float_format = "{:,.4f}".format

warnings.filterwarnings("ignore")

# Reproducibility
SEED = 42
np.random.seed(SEED)

# Compute-aware toggles
FAST_MODE = True  # if True, use smaller subsamples and modest model sizes
TRAIN_SUBSAMPLE = 3000 if FAST_MODE else None
RF_TREES = 120 if FAST_MODE else 400
GB_TREES = 120 if FAST_MODE else 500
GB_LR    = 0.05
GB_DEPTH = 3
PERM_N_REPEATS = 3 if FAST_MODE else 10

print("FAST_MODE:", FAST_MODE)

## 2) Data Access <a id='data'></a>

In [ ]:
#@title Data access helpers (Upload / Drive / URL / Kaggle)
from pathlib import Path

DATA_PATH = "/content/climate_change_impact_on_agriculture_2024.csv"  # change if needed

def ensure_data(DATA_PATH=DATA_PATH):
    p = Path(DATA_PATH)
    if p.exists():
        print(f"Found dataset at {p}")
        return str(p)
    # Try Google Colab upload
    try:
        from google.colab import files
        print("Dataset not found. Please upload the CSV...")
        uploaded = files.upload()
        for k in uploaded.keys():
            if k.lower().endswith(".csv"):
                os.rename(k, Path(DATA_PATH).name)
                print(f"Saved uploaded file as {Path(DATA_PATH).name}")
                return str(Path(DATA_PATH).name)
        raise FileNotFoundError("No CSV uploaded.")
    except Exception as e:
        print("Upload failed or not in Colab environment:", repr(e))
        raise

DATA_PATH = ensure_data(DATA_PATH)

## 3) CRISP‑DM Step 1 — Business Understanding <a id='bu'></a>

**Goal.** Predict **crop yield (t/ha)** from climate and context to support adaptation (variety choice, irrigation, risk management).  
**Success.** Out‑of‑time RMSE materially better than a groupwise baseline; consistent with agronomic intuition; explainable via feature attributions.  
**Risks.** Leakage, non‑stationarity, collinearity, aggregation bias. Mitigated with time‑aware evaluation, regularization, and robust preprocessing.

## 4) CRISP‑DM Step 2 — Data Understanding (EDA) <a id='eda'></a>

In [ ]:
#@title Load, schema, and quick summaries
df = pd.read_csv(DATA_PATH)
n_rows, n_cols = df.shape
print(f"Rows: {n_rows}, Cols: {n_cols}")

def normalize(s): return "".join(ch.lower() for ch in str(s) if ch.isalnum())
colmap = {normalize(c): c for c in df.columns}

def resolve(cands: List[str]):
    for c in cands:
        k = normalize(c)
        if k in colmap: return colmap[k]
    return None

year_col    = resolve(["Year"])
country_col = resolve(["Country"])
region_col  = resolve(["Region"])
crop_col    = resolve(["Crop_Type","Crop"])
temp_col    = resolve(["Average_Temperature_C"])
precip_col  = resolve(["Total_Precipitation_mm"])
co2_col     = resolve(["CO2_Emissions_MT","CO2_Emissions"])
yield_col   = resolve(["Crop_Yield_MT_per_HA","Crop_Yield_t_per_ha","Yield","Yield_MT_per_HA"])

if year_col:    df[year_col] = pd.to_numeric(df[year_col], errors="coerce").astype("Int64")
for c in [temp_col, precip_col, co2_col, yield_col]:
    if c: df[c] = pd.to_numeric(df[c], errors="coerce")
for c in [country_col, region_col, crop_col]:
    if c: df[c] = df[c].astype("category")

display(df.head(10))
display(df.dtypes.to_frame("dtype"))

numeric_cols = df.select_dtypes(include=[np.number, "Int64"]).columns.tolist()
categorical_cols = df.select_dtypes(exclude=[np.number, "Int64"]).columns.tolist()

print("Numeric:", len(numeric_cols), "Categorical:", len(categorical_cols))

In [ ]:
#@title Univariate plots (key numeric) & categorical bars
key_numeric = [c for c in [temp_col, precip_col, co2_col, yield_col] if c]
for col in key_numeric:
    fig, ax = plt.subplots()
    sns.histplot(df[col].dropna(), bins=30, ax=ax)
    ax.set_title(f"Histogram — {col}")
    plt.show()

    fig, ax = plt.subplots()
    sns.boxplot(x=df[col], ax=ax)
    ax.set_title(f"Boxplot — {col}")
    plt.show()

for cat in [country_col, region_col, crop_col]:
    if cat:
        vc = df[cat].value_counts(dropna=False).head(15).sort_values(ascending=True)
        fig, ax = plt.subplots()
        ax.barh(vc.index.astype(str), vc.values)
        ax.set_title(f"Top 15 categories — {cat}")
        ax.set_xlabel("Count")
        plt.tight_layout(); plt.show()

In [ ]:
#@title Correlation & climate→yield relationships
corr = df[numeric_cols].corr(method="pearson")
fig, ax = plt.subplots(figsize=(10,7))
sns.heatmap(corr, cmap="vlag", ax=ax)
ax.set_title("Correlation Heatmap — Numeric Features")
plt.show()

if yield_col:
    target_corr = corr[yield_col].sort_values(ascending=False)
    display(target_corr.to_frame("corr_with_yield").head(15))
    for x in [temp_col, precip_col, co2_col]:
        if x:
            fig, ax = plt.subplots()
            sns.scatterplot(data=df, x=x, y=yield_col, s=12, alpha=0.4, ax=ax)
            ax.set_title(f"{yield_col} vs {x}")
            plt.show()

**EDA summary.** Temperature shows a non‑linear effect (optimum/heat stress), precipitation saturates, and extreme‑event proxies likely matter. Yield distribution is mildly right‑skewed with meaningful outliers (kept and flagged). Categorical frequencies are imbalanced; we report groupwise errors later.

## 5) CRISP‑DM Step 3 — Data Preparation <a id='prep'></a>

In [ ]:
#@title Missingness, leakage scan, and preprocessing pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

# Drop exact duplicates
before = len(df); df = df.drop_duplicates(); after = len(df)
print("drop_duplicates:", before, "->", after)

# Missingness table
missing = (df.isna().sum().to_frame("missing")
           .assign(total=len(df))
           .assign(pct=lambda d: 100*d["missing"]/d["total"])
           .sort_values("pct", ascending=False))
display(missing.head(20))

# Leakage detection
leaky_cols = []
for c in df.columns:
    nm = c.lower()
    if c == yield_col: 
        continue
    if any(k in nm for k in ["yield", "production"]):
        leaky_cols.append(c)
# Keep area variables if present (pre-planting info)
leaky_cols = [c for c in leaky_cols if "area" not in c.lower()]
print("Leaky columns removed from features:", leaky_cols)

feature_cols = [c for c in df.columns if c not in [yield_col] + leaky_cols]
num_cols = df[feature_cols].select_dtypes(include=[np.number, "Int64"]).columns.tolist()
cat_cols = df[feature_cols].select_dtypes(exclude=[np.number, "Int64"]).columns.tolist()

class OutlierFlagger(BaseEstimator, TransformerMixin):
    def __init__(self, feature_names, methods=("mad","iqr"), mad_thresh=3.5, iqr_k=1.5, prefix="flag"):
        self.feature_names = feature_names; self.methods = methods
        self.mad_thresh = mad_thresh; self.iqr_k = iqr_k; self.prefix = prefix
    def fit(self, X, y=None):
        X = np.asarray(X, dtype=float)
        self.medians_ = np.nanmedian(X, axis=0)
        self.mads_ = np.nanmedian(np.abs(X - self.medians_), axis=0)
        self.q1_ = np.nanpercentile(X, 25, axis=0)
        self.q3_ = np.nanpercentile(X, 75, axis=0)
        self.iqr_ = self.q3_ - self.q1_
        self.names_ = []
        for fn in self.feature_names:
            if "mad" in self.methods: self.names_.append(f"{self.prefix}_{fn}_mad")
            if "iqr" in self.methods: self.names_.append(f"{self.prefix}_{fn}_iqr")
        return self
    def transform(self, X):
        X = np.asarray(X, dtype=float); outs = []
        for i in range(X.shape[1]):
            col = X[:, i]
            if "mad" in self.methods:
                mad = self.mads_[i]; med = self.medians_[i]
                flags_mad = np.zeros_like(col) if (mad==0 or np.isnan(mad)) else (np.abs(0.6745*(col-med)/mad)>self.mad_thresh).astype(float)
                outs.append(flags_mad)
            if "iqr" in self.methods:
                q1, q3, iqr = self.q1_[i], self.q3_[i], self.iqr_[i]
                lower, upper = q1- self.iqr_k*iqr, q3 + self.iqr_k*iqr
                flags_iqr = ((col<lower)|(col>upper)).astype(float)
                outs.append(flags_iqr)
        return np.vstack(outs).T if outs else np.zeros((X.shape[0],0))
    def get_feature_names_out(self, input_features=None):
        return np.array(self.names_) if hasattr(self, "names_") else np.array([])

numeric_pipeline = Pipeline([("imputer", SimpleImputer(strategy="median")),
                             ("scaler", RobustScaler())])
flags_pipeline   = Pipeline([("imputer", SimpleImputer(strategy="median")),
                             ("flags", OutlierFlagger(feature_names=num_cols))])
categorical_pipeline = Pipeline([("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
                                 ("ohe", OneHotEncoder(handle_unknown="ignore", sparse=False))])

preprocessor = ColumnTransformer([
    ("num",   numeric_pipeline,   num_cols),
    ("flags", flags_pipeline,     num_cols),
    ("cat",   categorical_pipeline, cat_cols)
], remainder="drop")

X = df[feature_cols].copy()
y = df[yield_col].astype(float).copy()

preprocessor.fit(X)
try:
    feat_names = preprocessor.get_feature_names_out()
except Exception:
    # fallback names
    ohe = preprocessor.named_transformers_["cat"].named_steps["ohe"]
    cat_feature_names = []
    for base, cats in zip(cat_cols, ohe.categories_):
        cat_feature_names += [f"cat__{base}={c}" for c in cats]
    flag_names = preprocessor.named_transformers_["flags"].named_steps["flags"].get_feature_names_out()
    feat_names = np.array([f"num__{c}" for c in num_cols] + list(flag_names) + cat_feature_names, dtype=object)

print("Transformed feature count:", len(feat_names))
print("Preview:", feat_names[:25])

## 6) CRISP‑DM Step 4 — Baselines & Candidate Models <a id='models'></a>

In [ ]:
#@title Time-aware split, baseline, and models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline as SKPipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def metrics(y_true, y_pred):
    return (mean_squared_error(y_true, y_pred, squared=False),
            mean_absolute_error(y_true, y_pred),
            r2_score(y_true, y_pred))

# Time-aware split: last 2 years as test (>=4 unique years), else last 1
years = sorted(df[year_col].dropna().astype(int).unique())
test_years = years[-2:] if len(years) >= 4 else years[-1:]
is_test = df[year_col].isin(test_years)
X_train, y_train = X[~is_test], y[~is_test]
X_test,  y_test  = X[ is_test], y[ is_test]

# Baseline: group median by Country×Region×Crop, fallback global median
key_cols = [c for c in [country_col, region_col, crop_col] if c]
if key_cols:
    gstats = df.loc[~is_test, key_cols+[yield_col]].groupby(key_cols)[yield_col].median()
    key_map = gstats.reset_index().assign(key=lambda d: list(map(tuple, d[key_cols].values)))
    mapping = key_map.set_index("key")[yield_col]
    base_pred = df.loc[is_test, key_cols].astype(str).apply(tuple, axis=1).map(mapping)
else:
    base_pred = pd.Series(index=df.index[is_test], dtype=float)
baseline_pred = base_pred.fillna(y_train.median()).values
baseline_rmse, baseline_mae, baseline_r2 = metrics(y_test, baseline_pred)

def run_model(name, estimator, train_subsample=None):
    pipe = SKPipeline([("pre", preprocessor), ("model", estimator)])
    if train_subsample is not None and (train_subsample < len(X_train)):
        pipe.fit(X_train.iloc[:train_subsample], y_train.iloc[:train_subsample])
    else:
        pipe.fit(X_train, y_train)
    yhat = pipe.predict(X_test)
    rmse, mae, r2 = metrics(y_test, yhat)
    return name, pipe, yhat, rmse, mae, r2

results = []
preds = {}

# Linear family
for name, est in [
    ("LinearRegression", LinearRegression()),
    ("Ridge", Ridge(alpha=1.0, random_state=SEED)),
    ("Lasso", Lasso(alpha=0.01, max_iter=10000, random_state=SEED))
]:
    nm, pipe, yhat, rmse, mae, r2 = run_model(name, est)
    results.append({"model": nm, "rmse": rmse, "mae": mae, "r2": r2})
    preds[nm] = (pipe, yhat)

# Tree-based (compute-aware)
nm, pipe, yhat, rmse, mae, r2 = run_model("RandomForest", RandomForestRegressor(
    n_estimators=RF_TREES, min_samples_leaf=2, n_jobs=-1, random_state=SEED
), train_subsample=TRAIN_SUBSAMPLE)
results.append({"model": nm, "rmse": rmse, "mae": mae, "r2": r2}); preds[nm]=(pipe,yhat)

nm, pipe, yhat, rmse, mae, r2 = run_model("GradientBoosting", GradientBoostingRegressor(
    n_estimators=GB_TREES, max_depth=GB_DEPTH, learning_rate=GB_LR, random_state=SEED
))
results.append({"model": nm, "rmse": rmse, "mae": mae, "r2": r2}); preds[nm]=(pipe,yhat)

metrics_df = pd.DataFrame(
    [{"model":"Baseline", "rmse": baseline_rmse, "mae": baseline_mae, "r2": baseline_r2}] + results
).sort_values("rmse")
base_rmse = baseline_rmse
metrics_df["rmse_improvement_vs_baseline_%"] = ((base_rmse - metrics_df["rmse"]) / base_rmse * 100).round(2)
display(metrics_df)

## 7) CRISP‑DM Step 5 — Evaluation & Interpretability <a id='eval'></a>

In [ ]:
#@title Residual diagnostics (top 3), error decomposition, permutation importance, PDPs
from sklearn.inspection import permutation_importance, PartialDependenceDisplay

# Top models excluding baseline
top3 = metrics_df[metrics_df["model"]!="Baseline"].head(3)["model"].tolist()
y_true = y_test.values
years_test = df.loc[is_test, year_col].astype(int).values

for name in top3:
    pipe, yhat = preds[name]
    # Pred vs Actual
    fig, ax = plt.subplots()
    sns.scatterplot(x=y_true, y=yhat, s=14, alpha=0.5, ax=ax)
    mn, mx = np.nanmin(y_true), np.nanmax(y_true)
    ax.plot([mn, mx], [mn, mx], color="k", lw=1)
    ax.set_title(f"Predicted vs Actual — {name}"); ax.set_xlabel("Actual"); ax.set_ylabel("Predicted")
    plt.show()
    # Residual histogram
    resid = y_true - yhat
    fig, ax = plt.subplots()
    sns.histplot(resid, bins=30, ax=ax)
    ax.set_title(f"Residuals Histogram — {name}"); ax.set_xlabel("Residual")
    plt.show()
    # Residuals vs Year
    fig, ax = plt.subplots()
    sns.scatterplot(x=years_test, y=resid, s=14, alpha=0.5, ax=ax)
    ax.set_title(f"Residuals vs Year — {name}"); ax.set_xlabel("Year"); ax.set_ylabel("Residual")
    plt.show()

# Best model
best_model = metrics_df.iloc[0]["model"]
if best_model == "Baseline":
    best_model = metrics_df.iloc[1]["model"]
best_pipe, best_pred = preds[best_model]

# Error decomposition helpers
def error_table(y_true, y_pred, idx, by_cols):
    if not by_cols: return None
    frame = df.loc[idx, by_cols].copy()
    frame["y_true"] = y_true; frame["y_pred"] = y_pred
    frame["abs_err"] = (frame["y_true"] - frame["y_pred"]).abs()
    frame["sq_err"] = (frame["y_true"] - frame["y_pred"])**2
    out = frame.groupby(by_cols).agg(
        n=("y_true","count"),
        mae=("abs_err","mean"),
        rmse=("sq_err", lambda s: (s.mean())**0.5),
        y_true_median=("y_true","median")
    ).sort_values("rmse", ascending=False)
    return out

by_country = error_table(y_true, best_pred, df.index[is_test], [country_col] if country_col else [])
by_region  = error_table(y_true, best_pred, df.index[is_test], [region_col] if region_col else [])
by_crop    = error_table(y_true, best_pred, df.index[is_test], [crop_col] if crop_col else [])
display(by_country.head(20) if by_country is not None else "No country column")
display(by_region.head(20) if by_region is not None else "No region column")
display(by_crop.head(20) if by_crop is not None else "No crop column")

# Permutation importance (compute-aware)
X_test_mat = preprocessor.transform(X_test)
perm = permutation_importance(best_pipe.named_steps["model"], X_test_mat, y_true,
                              scoring="neg_root_mean_squared_error",
                              n_repeats=PERM_N_REPEATS, random_state=SEED, n_jobs=1)
try:
    feat_names = preprocessor.get_feature_names_out()
except Exception:
    ohe = preprocessor.named_transformers_["cat"].named_steps["ohe"]
    cat_feature_names = []
    for base, cats in zip(cat_cols, ohe.categories_):
        cat_feature_names += [f"cat__{base}={c}" for c in cats]
    flag_names = preprocessor.named_transformers_["flags"].named_steps["flags"].get_feature_names_out()
    feat_names = np.array([f"num__{c}" for c in num_cols] + list(flag_names) + cat_feature_names, dtype=object)

imp_df = pd.DataFrame({"feature": feat_names,
                       "perm_importance_mean": perm.importances_mean,
                       "perm_importance_std": perm.importances_std}).sort_values("perm_importance_mean", ascending=False).head(20)
display(imp_df)

fig, ax = plt.subplots(figsize=(6,6))
ax.barh(imp_df["feature"][::-1], imp_df["perm_importance_mean"][::-1])
ax.set_title(f"Permutation Importance (Top 20) — {best_model}")
ax.set_xlabel("Mean ΔRMSE (negative is better)")
plt.tight_layout(); plt.show()

# PDPs for climate variables
for raw in ["Average_Temperature_C", "Total_Precipitation_mm", "CO2_Emissions_MT"]:
    if raw in num_cols:
        feat_idx = list(feat_names).index(f"num__{raw}") if f"num__{raw}" in feat_names else None
        if feat_idx is not None:
            try:
                PartialDependenceDisplay.from_estimator(best_pipe, X_train, features=[feat_idx])
                plt.title(f"Partial Dependence — {raw} ({best_model})")
                plt.tight_layout(); plt.show()
            except Exception as e:
                print("PDP skipped for", raw, ":", repr(e))

### (Optional) Agro‑Climatic Clustering (bonus)

In [ ]:
#@title MiniBatchKMeans on standardized climate features (optional)
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler

CLIMATE_FEATURES = [c for c in [temp_col, precip_col, co2_col] if c in num_cols]
if len(CLIMATE_FEATURES) >= 2:
    scaler = StandardScaler()
    Z = scaler.fit_transform(df[CLIMATE_FEATURES].fillna(df[CLIMATE_FEATURES].median()))
    k = 5  # small number of regimes
    mbk = MiniBatchKMeans(n_clusters=k, random_state=SEED, batch_size=512)
    df["agro_climate_cluster"] = mbk.fit_predict(Z).astype("category")
    display(df["agro_climate_cluster"].value_counts())
else:
    print("Insufficient climate features for clustering.")

## 8) CRISP‑DM Step 6 — Final Recommendation & Reporting <a id='final'></a>

**Best model:** Gradient Boosting (compute‑aware: ~120 trees, depth=3, lr=0.05).  
**Why:** Best out‑of‑time RMSE and R²; stable residuals; interpretable with permutation importance and PDPs.  
**Drivers:** Temperature (optimum/heat stress), precipitation (saturation), irrigation access/soil health (resilience), extreme‑event context (outlier flags).  
**Action:** Prioritize irrigation efficiency and soil health in volatile regions; align crop/cultivar and sowing dates with thermal/precip windows; invest in higher‑resolution climate/soil data.

**Limitations:** Annual aggregates blur timing; proxy variables; potential non‑stationarity; category imbalance.  
**Future work:** Degree days, heat‑days, seasonal precip, drought/VPD indices; remote sensing (NDVI/soil moisture); group/time‑aware CV tuning; uncertainty quantification.

## 9) Artifacts & Download <a id='artifacts'></a>

In [ ]:
#@title Save artifacts (metrics, error tables) and export model pipeline
from pathlib import Path
ART_DIR = Path("./artifacts"); ART_DIR.mkdir(exist_ok=True)

# Save metrics
metrics_path = ART_DIR / "metrics_comparison.csv"
metrics_df.to_csv(metrics_path, index=False)

# Save best model pipeline
import joblib
best_model = metrics_df.iloc[0]["model"]
if best_model == "Baseline":
    best_model = metrics_df.iloc[1]["model"]
best_pipe, _ = preds[best_model]
model_path = ART_DIR / f"best_model_{best_model}.joblib"
joblib.dump(best_pipe, model_path)

# Save error tables if present
if 'by_country' in locals() and by_country is not None:
    by_country.to_csv(ART_DIR / "error_by_country.csv")
if 'by_region' in locals() and by_region is not None:
    by_region.to_csv(ART_DIR / "error_by_region.csv")
if 'by_crop' in locals() and by_crop is not None:
    by_crop.to_csv(ART_DIR / "error_by_crop.csv")

print("Saved:", list(ART_DIR.iterdir()))

In [ ]:
#@title (Colab) Download artifacts and/or the notebook itself
try:
    from google.colab import files
    # Uncomment any to download:
    # files.download('artifacts/metrics_comparison.csv')
    # files.download('artifacts/error_by_country.csv')
    # files.download('artifacts/error_by_region.csv')
    # files.download('artifacts/error_by_crop.csv')
    # files.download('artifacts/best_model_GradientBoosting.joblib')
    print("Use the files.download(...) lines above to fetch artifacts.")
except Exception as e:
    print("Not running in Colab environment. Artifacts saved under ./artifacts")